In [550]:
from src.data_processing import data_processing, data_dropping
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, Lasso, LinearRegression
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, make_scorer, accuracy_score, mean_squared_error
from nltk.stem import WordNetLemmatizer as Lemmatizer
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

In [494]:
with open('data/predictor.p', 'rb') as file:
    predictor = pickle.load(file)
with open('data/target.p', 'rb') as file:
    target = pickle.load(file)

In [495]:
pred = data_processing(data_dropping(predictor))

In [496]:
pred = pred[['existing_stories', 'existing_use', 'proposed_use', 'description']]

In [14]:
pred['has_existing_stories'] = (pred['existing_stories'].fillna(0) > 0).astype(int)

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [497]:
pred[['existing_use', 'proposed_use', 'description']] = pred[['existing_use', 'proposed_use', 'description']].fillna('-').astype(str)

In [498]:
pred['text'] = pred[['existing_use', 'proposed_use', 'description']].apply(lambda x: ' '.join(x), axis = 1)

In [581]:
float('29')

29.0

In [474]:
with open('data/pred_trimmed.p', 'wb') as file:
    pickle.dump(pred, file)

In [499]:
text = pred['text'].values

In [519]:
y_full = (target['fire_rate_after'] - target['fire_rate_before']).values

In [582]:
def int_able(str):
    try:
        float(str)
    except:
        return False
    return True

In [583]:
text = np.array([' '.join([Lemmatizer.lemmatize('n', word.lower()) for word in doc.split() if not int_able(word)]) for doc in text])

In [478]:
text_lemmatized = text

In [455]:
n = 50000
sample_idx = np.random.choice(len(pred), n)

In [565]:
X = text[sample_idx]
y = y_full[sample_idx]

In [439]:
train = np.full(n, False)
train_idx = np.random.choice(n, int(.75*n), replace = False)
for idx in train_idx:
    train[idx] = True
test = ~train

In [563]:
vec = TfidfVectorizer(stop_words = 'english')

In [566]:
X = vec.fit_transform(X)

In [551]:
model = GradientBoostingRegressor()

In [567]:
model.fit(X[train],y[train])

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [568]:
model.score(X[test], y[test])

-0.07151856906067922

In [569]:
mean_squared_error(y[test], model.predict(X[test]))

140.022278473845

In [576]:
vocab_idx[10774]

'201609087093'

In [579]:
model.feature_importances_.argsort()

array([    0, 23578, 23577, ..., 10774, 14949, 14974], dtype=int64)

In [526]:
roc_auc_score(y[test], model.predict_proba(X[test])[:,1])

AttributeError: 'Lasso' object has no attribute 'predict_proba'

In [506]:
def f1(y_test, y_pred_prob, threshold):
    y_pred = (y_pred_prob > threshold).astype(int)
    return f1_score(y_test, y_pred)

In [507]:
current_best = 0
best_i = 0
for i in range(1000):
    f = f1(y[test], model.predict_proba(X[test])[:,1], i/1000)
    if f > current_best:
        current_best = f
        best_i = i
    

C:\Users\pengj\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [508]:
f1(y[test], model.predict_proba(X[test])[:,1], best_i/1000)

0.43174820390010266

In [509]:
vec.vocabulary_;

In [572]:
vocab_idx = {v: k for k, v in vec.vocabulary_.items()}

In [537]:
important_words = {}

In [539]:
for i, coef in enumerate(model.coef_):
    if coef > 0:
        important_words[vocab_idx[i]] = coef

In [540]:
c = Counter(important_words)

In [543]:
important_words

{}

In [514]:
top10_decrease_2 = c.most_common(10)

In [468]:
top10_decrease_1

[('cti', 6.555634496222449),
 ('201504214211', 5.459267580754549),
 ('attach', 4.15891572645569),
 ('3c', 3.6917141823836754),
 ('contractor', 3.483833784581392),
 ('plansnan', 3.2166330652688555),
 ('smoke', 3.1106596356006744),
 ('input', 2.7451690269594837),
 ('substitute', 2.7419791142035774),
 ('closetnan', 2.6672226268668453)]

In [515]:
top10_decrease_2

[('cti', 8.568333416450177),
 ('105', 4.519257013734376),
 ('coutnertop', 4.345246902269198),
 ('oti', 3.3374166992264547),
 ('drinking', 3.262354378056696),
 ('screens', 3.087552444844313),
 ('density', 2.990672860273146),
 ('addressable', 2.9302821775207684),
 ('winder', 2.9110521360872648),
 ('fluorescent', 2.8903632859330872)]

In [381]:
important_words_4 = important_words

In [392]:
important_words;

In [415]:
top10_2 = c.most_common(10)

In [401]:
top10_1 = c.most_common(10)

In [433]:
top10_3

[('traps', 9.066135535601088),
 ('exh', 5.236541100818746),
 ('265', 4.812601390528742),
 ('maher', 4.730539830504769),
 ('tags', 4.297665411723898),
 ('geotech', 4.20148706744233),
 ('901', 4.104538395668117),
 ('9884', 4.058352578295828),
 ('notifier', 3.8099096402798533),
 ('144', 3.680255661760993)]

In [416]:
top10_2

[('traps', 7.005050918380217),
 ('maher', 4.692009685187687),
 ('forms', 3.636798151040242),
 ('ref', 3.2956199551363703),
 ('evacuation', 2.8559641655760863),
 ('apt', 2.763896991448432),
 ('assmbly', 2.759404362814315),
 ('3884sf', 2.631019912856813),
 ('suite', 2.5307707835609),
 ('unit', 2.5235550187500255)]

In [417]:
top10_1

[('traps', 6.190488019758687),
 ('maher', 4.546629916650752),
 ('uprights', 3.1050460402905613),
 ('daycare', 2.8857078892696255),
 ('sounders', 2.7342078862780332),
 ('altered', 2.622869978271953),
 ('evacuation', 2.6174622302659016),
 ('washer', 2.552283019923069),
 ('arch', 2.4326521113122124),
 ('chap', 2.366688547050922)]

In [516]:

for i, row in pred[pred['description'].apply(lambda x: ' CTI-' in x)].reset_index().iterrows():
    print(row['description'])
    if i > 10:
        break

SPACE# 422. CTI-1  4TH FLOOR   EL  ME  LS  SP. TENANT IMPROVEMENT/STORE FRONT  SPRINKLER MODIFICATION  PAINT  METAL FIXTURES AND LIGHTING/
STRAITS CAFE  SPACE# 597. CTI-1  4TH  EL  ME  LS  SP.
BATH AND BODY  SPACE 357  3RD FL  CTI-2  EL  ME  LS  SP.
BATH AND BODY  SPACE 357  3RD FL  CTI-2  EL  ME  LS  SP.
BATH AND BODY  SPACE 357  3RD FL  CTI-2  EL  ME  LS  SP.
BATH AND BODY  SPACE 357  3RD FL  CTI-2  EL  ME  LS  SP.
JUICY COUTURE  SPACE# 137  CTI-2  1ST FLOOR  EL  ME.
BUCKHORN  BASEMENT  SPACE# FE 5. CTI-2  EL  ME.
'BRISTOL FARMS'  SPACE# 010  CTI-2  BASEMENT  EL  ME.
'BRISTOL FARMS'  SPACE# 010  CTI-2  BASEMENT  EL  ME.
PASTA MOTO  FE 6. CTI-1  BART LEVEL  EL  ME  LS  SP.
ANDALE  SPACE# FE-7. CTI-2  BART LEVEL  LS  SP.
